In [2]:
import json
import os
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import datetime
import sys
sys.path.insert(0, 'E:\\StuDY\\Keys')
from spotify_cred import CLIENT_SECRET, CLIENT_ID

In [5]:
def find_genre(original_list):
    new_list = []
    for item in original_list:
        if " " in item:
            new_list.extend(item.split())
        else:
            new_list.append(item)

    counts = {}
    for word in new_list:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    most_common_word = max(counts, key=counts.get)
    return most_common_word

In [13]:
#get current date
now = datetime.datetime.now()
year = '{:02d}'.format(now.year)
month = '{:02d}'.format(now.month)
day = '{:02d}'.format(now.day)
day_month_year = '{}-{}-{}'.format(year, month, day)

# Step 2: Authenticate with the Spotify API
client_credentials_manager = SpotifyClientCredentials(
    client_id=CLIENT_ID, 
    client_secret=CLIENT_SECRET
)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Step 3: Get the top 10 tracks for each country
countries = ["United States", "India", "Brazil", "Mexico", "United Kingdom", "Germany", "Sweden"]

country_codes = ["USA", "India", "Brazil", "Mexico", "UK", "Germany", "Sweden"]

country_df = pd.DataFrame(
    {'country': countries,
     'country_code': country_codes
    })
sp.search(q=f'top 50 India', type='playlist')['playlists']

{'href': 'https://api.spotify.com/v1/search?query=top+50+India&type=playlist&offset=0&limit=10',
 'items': [{'collaborative': False,
   'description': 'Top 50 - India | Top 50',
   'external_urls': {'spotify': 'https://open.spotify.com/playlist/2WKT9LWsW6YD8OPc70mTXF'},
   'href': 'https://api.spotify.com/v1/playlists/2WKT9LWsW6YD8OPc70mTXF',
   'id': '2WKT9LWsW6YD8OPc70mTXF',
   'images': [{'height': None,
     'url': 'https://i.scdn.co/image/ab67706c0000bebb6a66040158a33b7bf774d5a0',
     'width': None}],
   'name': 'Top 50 India',
   'owner': {'display_name': 'India Top 50',
    'external_urls': {'spotify': 'https://open.spotify.com/user/mvuoctmhex6vvn8496s2i85kq'},
    'href': 'https://api.spotify.com/v1/users/mvuoctmhex6vvn8496s2i85kq',
    'id': 'mvuoctmhex6vvn8496s2i85kq',
    'type': 'user',
    'uri': 'spotify:user:mvuoctmhex6vvn8496s2i85kq'},
   'primary_color': None,
   'public': None,
   'snapshot_id': 'MjQ5OSxjOTQ5MGZmMjU2MGE3ZGExOTdmZTY5ZDJlYTBkYjVmZTY4NmRkZDFi',
   'trac

In [14]:
#get current date
now = datetime.datetime.now()
year = '{:02d}'.format(now.year)
month = '{:02d}'.format(now.month)
day = '{:02d}'.format(now.day)
day_month_year = '{}-{}-{}'.format(year, month, day)

# Step 2: Authenticate with the Spotify API
client_credentials_manager = SpotifyClientCredentials(
    client_id=CLIENT_ID, 
    client_secret=CLIENT_SECRET
)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Step 3: Get the top 10 tracks for each country
countries = ["United States", "India", "Brazil", "Mexico", "United Kingdom", "Germany", "Sweden"]

country_codes = ["USA", "India", "Brazil", "Mexico", "UK", "Germany", "Sweden"]
country_df = pd.DataFrame(
    {'country': countries,
     'country_code': country_codes
    })

tracks = {}
country_dfs = []
for country in country_codes:
    df = []
    audio_features = []
    try:
        #sometimes, it does not have a playlist 
        playlist_id = sp.search(q=f'top 50 {country}', type='playlist')['playlists']['items'][0]['id']
        playlist_tracks = sp.playlist_tracks(playlist_id, limit=50)['items']
    except IndexError as e:
        print('no {} playlist').format(country)
        continue
    for r in playlist_tracks:
        song_id = r['track']['id']
        song_name = r['track']['name']

        song_url = r['track']['external_urls']['spotify']

        song_popularity = r['track']['popularity']

        album_id = r['track']['album']['id']

        album_name = r['track']['album']['name']
        album_url = r['track']['album']['external_urls']['spotify']
        artist_id = r['track']['album']['artists'][0]['id']
        track_preview = r['track']['preview_url']
        try:
            album_image= r['track']['album']['images'][0]['url']
        except:
            try:
                album_image = 'Null'
            except:
                print('both code failed')
        artist_name = r['track']['album']['artists'][0]['name']
        results = sp.search(q = artist_name, type = 'artist')['artists']['items'][0]
        
        artist_type = results['genres']
        artist_followers = results['followers']['total']
        artist_popularity = results['popularity']
        artist_url = r['track']['album']['artists'][0]['external_urls']['spotify']

        try:
            artist_image = results['images'][0]['url']
        except:
            try:
                artist_image = 'Null'
            except:
                print('both code failed')
        record_date = day_month_year
        if len(artist_type) != 0 :
            genre = find_genre(artist_type)
        else:
            genre = 'Null'
        attribute = {'track_id': song_id,'track_name':song_name,'track_url':song_url,
                    'track_popularity':song_popularity, 
                    'album_id':album_id , 'album_name': album_name ,
                    'album_url':album_url, 'album_image': album_image, 'track_preview' : track_preview,
                    'artist_id':artist_id , 'artist_name': artist_name, 'artist_url':artist_url, 'artist_image' :artist_image,
                    'artist_genre':genre, 'artist_followers': artist_followers, 'artist_popularity':artist_popularity,'record_date':record_date,
                    'Country': country, 'playlist_id': playlist_id
            }

        df.append(attribute)
        feature = sp.audio_features(song_url)[0]
        audio_features.append(feature)
    # remove some attributes    
    rem_list = ['id', 'track_href','uri','type','analysis_url']
    for row in audio_features:
        for key in rem_list:
            if len(row) != 0:
                if key in row:
                    del row[key]
    final_df = []
    for i in range(len(audio_features)):
        res = {**df[i],**audio_features[i]}
        final_df.append(res)                
    country_dfs.append(pd.DataFrame(final_df))
track_by_country = pd.concat(country_dfs, ignore_index=True)
track_by_country.columns

Index(['track_id', 'track_name', 'track_url', 'track_popularity', 'album_id',
       'album_name', 'album_url', 'album_image', 'track_preview', 'artist_id',
       'artist_name', 'artist_url', 'artist_image', 'artist_genre',
       'artist_followers', 'artist_popularity', 'record_date', 'Country',
       'playlist_id', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature'],
      dtype='object')